In [1]:
# BIBLIOTECAS
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import chi2, SelectKBest
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score

# CARGAR DATOS
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv", sep=";")

# ANALISIS Y LIMPIEZA DEL DF
dfd = df.drop_duplicates()

# NORMALIZACIÓN DE LOS DATOS Y CAMBIO DE CUALITATIVOS A CUANTITATIVOS
for column in ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "day_of_week", "poutcome", "y"]:
    dfd[column + "_n"] = pd.factorize(dfd[column])[0]

# TRAIN Y TEST
num_variables = [col + "_n" for col in ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "day_of_week", "poutcome", "age", "duration", "campaign", "pdays", "previous", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"]]
X = dfd[num_variables] 
y = dfd["y_n"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ESCALADO CON MINMAXSCALER
scaler = MinMaxScaler()
X_train_esc = scaler.fit_transform(X_train)
X_test_esc = scaler.transform(X_test)

# SELECCIÓN DE CARACTERÍSTICAS
selection_model = SelectKBest(chi2, k=5)
X_train_sel = selection_model.fit_transform(X_train_esc, y_train)
X_test_sel = selection_model.transform(X_test_esc)

# OPTIMIZACIÓN
model = LogisticRegression(max_iter=500)

# Define la cuadrícula de hiperparámetros
hiperparametros = {"C": [0.0001, 0.001, 0.01, 0.1, 1, 10], "penalty": ["l1", "l2", "elasticnet", None], "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"], 'max_iter': [100, 200, 500]}
grid = GridSearchCV(model, hiperparametros, scoring="accuracy", cv=5)

# Ajustar la cuadrícula a los datos seleccionados
grid.fit(X_train_sel, y_train)

# Mejor hiperparámetro
print(f"Mejores hiperparámetros: {grid.best_params_}")

# Modelo optimizado
model_op = LogisticRegression(**grid.best_params_, max_iter=500)
model_op.fit(X_train_sel, y_train)

# Comprobar cómo funciona el modelo optimizado
y_pred = model_op.predict(X_test_sel)
print(f"Accuracy del modelo optimizado: {accuracy_score(y_test, y_pred):.7f}")

# Matriz de confusión
matriz_op = confusion_matrix(y_test, y_pred)
cm_df_op = pd.DataFrame(matriz_op)

plt.figure(figsize=(3, 3))
sns.heatmap(cm_df_op, annot=True, fmt="d")
plt.title("Matriz de Confusión - Modelo Optimizado")
plt.show()


/tmp/ipykernel_11251/2333658979.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfd[column + "_n"] = pd.factorize(dfd[column])[0]
/tmp/ipykernel_11251/2333658979.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfd[column + "_n"] = pd.factorize(dfd[column])[0]
/tmp/ipykernel_11251/2333658979.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

KeyError: "['age_n', 'duration_n', 'campaign_n', 'pdays_n', 'previous_n', 'emp.var.rate_n', 'cons.price.idx_n', 'cons.conf.idx_n', 'euribor3m_n', 'nr.employed_n'] not in index"